<a href="https://colab.research.google.com/github/klklgooa/MachineLearning/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

정형 데이터를다루는 데 가장 뛰어난성과를 내는 알고리즘
대다수의 앙상블 학습 알고리즘은 결정 트리를 기반으로 함

정형데이터 vs 비정형데이터

정형데이터적합
k최근저이웃, 선형회귀,릿지, 랏쏘

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 랜덤포레스트

앙상블 학습 중 가장 유명, 안정적인성능을 제공
1.훈련하기 위한 데이터를 랜덤하게 만듬
입력한 훈련 데이터에서 랜덤하게 샘플을 추출
중복된 샘플을 추출할 수 있음
이렇게 만들어진 샘플을 부트스트랩 샘플이라고 함
기본적으로 훈련 세트의 크기와 동일하게 설정됨

2.부트스트랩 샘플로 결정 트리를 훈련
각 노드를 분할 할 때 전체 특성 중에서 일부 특성을 무작위로 고르 다음 이 중에서 최선의 분할을 찾음.
RandomForestLcassifier

3.정해진 수 만큼 위 과정을 반복
기본적으로 100개의 결정 트리를 훈련.
분류일 때는 각 트리의 클래스별 확률을 평균 높은 클래스를 예측으로 결정
회귀일 때는 각 트리의 에측을 평균함
랜덤하게 선택한 샘플과 특성을 사용하기때문에 훈련세트에과대적합되는 것을 막아주고 검증세트와 테스트 세트에서안정적인 성능을 얻을 수 있음
랜덤 포레스트는 누락된값이 있어도 처리가 가능하다는 장점이 있음

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_) #특성 중요도 출력

[0.23167441 0.50039841 0.26792718]


특성중요도가 일부 상승함. 하나의 특성에 과도하게 집중하지 않음.

obb_score랜덤하게 부트스트랩 샘플을선택하기 때문에....

In [5]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

랜덤포레스트와 비슷.
부트스트랩 샘플을 사용하지 않고 각 결정트리를 만들 때 전체 훈련 세트를 사용 노드를 분할 할 때 가장좋은 분할을 찾지 않고 랜덤으로 사용.

In [6]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [7]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


single - 원래하던 방식
bagging - 여러개의 모델훈련, 각 모델 예측을 활용
boosting - 모델만들고 보안한 모델을 만들고 반복.
## 그레이디언트 부스팅

깊이가 얕은 결정 트리를 사용하여 이전 트리읭차를 보완하는 방식. 사이킷런에서는 기본적으로깊이가 3인 결정 트리 100개를 사용
깊이가 얕은 결정 트리를 사용하므로 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음.

경사하강법과 비교하면
결정트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동함.


https://www.youtube.com/watch?v=GciPwN2cde4

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [9]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [10]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 부스팅

정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기많은 알고리즘
입력 특성을 256개의 구간으로 나누므로 노드를 분할할때 최적의 분할을 매우 빠르게 찾을 수 있음.

In [11]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [12]:
from sklearn.inspection import permutation_importance
#특성을 하나씩랜덤하게 섞어서 모델의 성능 변화를 관찰하여 어떤 특성이 중요한지 계산함

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


테스트 세트에 대한 특성 중요도 계산.

In [13]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [14]:
hgb.score(test_input, test_target)

0.8723076923076923

다른 곳에서 그레이디언트 부스팅 알고리즘을 구현하는 경우

#### XGBoost

In [15]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


#### LightGBM

In [16]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
